In [4]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
from collections import namedtuple
from collections import defaultdict
from collections import Counter
import itertools
import json
import random
import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.metrics import confusion_matrix
import boto3
import sagemaker
from urllib.parse import urlparse


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
BUCKET = 'white-100'

# Make sure the bucket is in the same region as this notebook.
role = sagemaker.get_execution_role()
region = boto3.session.Session().region_name
s3 = boto3.client('s3')
bucket_region = s3.head_bucket(Bucket=BUCKET)['ResponseMetadata']['HTTPHeaders']['x-amz-bucket-region']
assert bucket_region == region, "You S3 bucket {} and this notebook need to be in the same region.".format(BUCKET)

ValueError: Must setup local AWS configuration with a region supported by SageMaker.

In [6]:
# Specify ARNs for resources needed to run an image classification job.

prehuman_arn = 'arn:aws:lambda:us-east-2:522254924758:function:serverlessrepo-aws-sagema-GtRecipePreHumanTaskFunc-1L5Z2249KDMHU'
acs_arn      = 'arn:aws:lambda:us-east-2:522254924758:function:serverlessrepo-aws-sagema-GtRecipeAnnotationConsol-1JQ1AEBC0TNB3'
workteam_arn = 'arn:aws:sagemaker:us-east-2:394669845002:workteam/public-crowd/default'

In [11]:
task_description = 'Please label each body part of interest'
task_keywords = ['pose', 'keypoints','animal']
task_title = task_description
job_name = '{}-'.format(BUCKET) + str(int(time.time()))

human_task_config = {
      "AnnotationConsolidationConfig": {
        "AnnotationConsolidationLambdaArn": acs_arn,
      },
      "PreHumanTaskLambdaArn": prehuman_arn,
      "MaxConcurrentTaskCount": 200, # 200 images will be sent at a time to the workteam.
      "NumberOfHumanWorkersPerDataObject": 6, # 3 separate workers will be required to label each image.
      "TaskAvailabilityLifetimeInSeconds": 12*60*60, # seconds to complete all pending tasks.
      "TaskDescription": task_description,
      "TaskKeywords": task_keywords,
      "TaskTimeLimitInSeconds": 300, # Each image must be labeled within 5 minutes.
      "TaskTitle": task_title,
      "UiConfig": {
        "UiTemplateS3Uri": 's3://white-mouse-test/final_keypoint_unique_label_white.template',
      }
    }


human_task_config["PublicWorkforceTaskPrice"] = {
    "AmountInUsd": {
       "Dollars": 0,
       "Cents": 6,
       "TenthFractionsOfACent": 0,
    }
} 
human_task_config["WorkteamArn"] = workteam_arn


ground_truth_request = {
        "InputConfig" : {
          "DataSource": {
            "S3DataSource": {
              "ManifestS3Uri": 's3://{}/white_mouse_100.manifest'.format(BUCKET),
            }
          },
          "DataAttributes": {
            "ContentClassifiers": [
              "FreeOfPersonallyIdentifiableInformation",
              "FreeOfAdultContent"
            ]
          },  
        },
        "OutputConfig" : {
          "S3OutputPath": 's3://{}-output/'.format(BUCKET),
        },
        "HumanTaskConfig" : human_task_config,
        "LabelingJobName": job_name,
        "RoleArn": role, 
        "LabelAttributeName": "annotatedResult",
    }

sagemaker_client = boto3.client('sagemaker')
sagemaker_client.create_labeling_job(**ground_truth_request)

{'LabelingJobArn': 'arn:aws:sagemaker:us-east-2:522254924758:labeling-job/white-100-1561435860',
 'ResponseMetadata': {'RequestId': 'e3fcd117-a2f8-484e-bde0-69b2b7a54269',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e3fcd117-a2f8-484e-bde0-69b2b7a54269',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '95',
   'date': 'Tue, 25 Jun 2019 04:10:59 GMT'},
  'RetryAttempts': 0}}